In [1]:
%%html
<style>
.reveal .rendered_html table, .reveal .rendered_html th,.reveal .rendered_html tr,.reveal .rendered_html td {
     font-size: 100%;
}
</style>

# Introduction To Recommender Systems

![title](resources/user_to_user.jpg)

# Goal of recommender systems

> *to predict the "rating" or "preference" a user would give to an (unseen) item*<br>

<center><img src="https://media.giphy.com/media/3pzLJifxEvLpe/giphy.gif" width="1000"></center>

<center><img src="resources/Example.png" width="800"></center>

What rate do you guess that user 1 gives item 3?

##  Technology evolution of recommenders
<center><img src="resources/history.png" height="500"></center>

## Content-based recommendation
<center><img src="resources/contentbased.png" height="500"></center>

## Collaborative Filtering recommendation engine
<center><img src="resources/CF.png" width="800"></center>

##  Technology evolution of recommenders
<center><img src="resources/history.png" height="500"></center>

##  Collaborative Filtering Methods
- User-to-User Based: Users who are similar to you also liked ...
- Item-to-Item Based: Users who liked this item also liked... 


## User-to-User based
<center><img src="resources/user-based.png" height="700"></center>

## Item-to-Item based
<center><img src="resources/item-based.png" height="700"></center>

# Typical User-Item Matrix 
- The matrix contains either explicit (ratings) or implicit feedback (bought/clicked/viewed)
<img src="resources/BLOG_CCA_8.png" width="1000">


# User-to-User Simularity
<center><img src="resources/BLOG_CCA_11.png" width="600"></center>

# Item-to-Item Simularity

<center><img src="resources/BLOG_CCA_10.png" width="600"></center>

## Similarity metrics used in recommendation engines
* Cosine distance <br>
$
d^{cosine}(\mathbf{x}_a,\mathbf{x}_b)
 = \frac{ \mathbf{x}_a \cdot \mathbf{x}_b }{ \left \| \mathbf{x}_a \right \| \left \| \mathbf{x}_b \right \| }
 = \frac{ \sum_{i=1}^k x_{a,i}\cdot x_{b,i} }{\sqrt{\sum_{i=1}^k x_{a,i}^2 } \sqrt{\sum_{i=1}^k x_{b,i}^2} }
$
* Pearson correlation distance<br>
$
d^{pearson}(\mathbf{x}_a,\mathbf{x}_b)
 = \frac{ \sum_{i=1}^k (x_{a,i} - \bar{\mathbf{x}}_a) \cdot (x_{b,i} - \bar{\mathbf{x}}_b) }{\sqrt{\sum_{i=1}^k (x_{a,i} - \bar{\mathbf{x}}_a)^2 } \sqrt{\sum_{i=1}^k (x_{b,i} - \bar{\mathbf{x}}_b)^2} }
$
* Euclidean distance
* Manhattan distance
* Jaccard distance


# Hands on a transaction dataset

## Read in the transactional data

In [2]:
import numpy as np, pandas as pd
transactions = pd.read_pickle('data/transactions.pkl')

n_users = transactions.user_id.unique().shape[0]
n_items = transactions.product_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of products = ' + str(n_items))
print(f'Number of rows in transactional data = {transactions.shape[0]}')

Number of users = 5000 | Number of products = 1000
Number of rows in transactional data = 453836


In [3]:
transactions.sort_values(by=['user_id','order_id','product_id'],inplace=True)
transactions = transactions[['user_id','order_id','product_id','product_name']]
transactions.head(20)

,user_id,order_id,product_id,product_name
4698597,18,1020460,21019,Roasted Red Pepper Hummus
2762349,18,1020460,21137,Organic Strawberries
10491102,18,1020460,24830,Organic Romaine
8142399,18,1020460,26209,Limes
11926926,18,1020460,34126,Organic Italian Parsley Bunch
19160225,18,1020460,38739,Hass Avocado
15174494,18,2461523,5450,Small Hass Avocado
4483330,18,2461523,8518,Organic Red Onion
587782,18,2461523,17461,Air Chilled Organic Boneless Skinless Chicken ...
2955579,18,2461523,21137,Organic Strawberries


<center><img src="resources/simplify.jpg" width="600"></center>

### During this workshop, we do not look at frequency. Only, whether a user bought an item or not.

In [4]:
transactions = transactions.drop('order_id',axis=1).drop_duplicates() 
# only keep single interactions

In [5]:
transactions.sort_values(by='user_id').head()

,user_id,product_id,product_name
4698597,18,21019,Roasted Red Pepper Hummus
2402823,18,31717,Organic Cilantro
7098624,18,5876,Organic Lemon
20103043,18,4461,Organic Raw Unfiltered Apple Cider Vinegar
17431079,18,22888,90% Lean Ground Beef


# Normalize user_id
As we go to a sparse representation of the matrix later, we make the id's of both users and products range from 0 to n_users and n_items respectively. 

In [6]:
from resources.tools import Normalizer
user_normalizer = Normalizer(transactions.user_id.unique())
transactions['user'] = user_normalizer.apply(transactions.user_id)

item_normalizer = Normalizer(transactions.product_id.unique())
transactions['item'] = item_normalizer.apply(transactions.product_id)

In [7]:
transactions.sort_values(by=['user','item'],inplace=True)
transactions.head(50)

,user_id,product_id,product_name,user,item
20103043,18,4461,Organic Raw Unfiltered Apple Cider Vinegar,0,72
15174494,18,5450,Small Hass Avocado,0,101
7098624,18,5876,Organic Lemon,0,115
4483330,18,8518,Organic Red Onion,0,167
587782,18,17461,Air Chilled Organic Boneless Skinless Chicken ...,0,343
4698597,18,21019,Roasted Red Pepper Hummus,0,424
2762349,18,21137,Organic Strawberries,0,425
17431079,18,22888,90% Lean Ground Beef,0,461
10491102,18,24830,Organic Romaine,0,500
8142399,18,26209,Limes,0,532


# Split the ratings for train and test

In [8]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(transactions, test_size=0.2,random_state=3)

# Create user-item matrix from set of ratings

In [9]:
# We fill in the user item matrix for train and test set. 
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line.user, line.item] = 1

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line.user, line.item] = 1

# Determine how sparse/dense the train matrix is

In [10]:
density = train_data_matrix.mean()*100
print('The train_data_matrix is filled for %3.2f%%'%(density))

The train_data_matrix is filled for 2.45%


# Calculate cosine simularity matrices from the train set
$
s_u^{cos}(i_m,i_b)
 = \frac{ i_m \cdot i_b }{ \left \| i_m \right \| \left \| i_b \right \| }
 = \frac{ \sum x_{a,m} x_{a,b} }{ \sqrt{ \sum x_{a,m}^2 \sum x_{a,b}^2 } }
$

$
s_u^{cos}(u_k,u_a)
 = \frac{ u_k \cdot u_a }{ \left \| u_k \right \| \left \| u_a \right \| }
 = \frac{ \sum x_{k,m}x_{a,m} }{ \sqrt{\sum x_{k,m}^2\sum x_{a,m}^2} }
$

Hint: checkout pairwise_distances from sklearn 

In [11]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = 1 - pairwise_distances(train_data_matrix,  metric='cosine')
item_similarity = 1 - pairwise_distances(train_data_matrix.T, metric='cosine')

# What 10 items are most similar to 'Organic Hass Avocado'?

<center><img src="resources/avocado.jpg" width="600"></center>

In [12]:
product_lookup = transactions[['product_id','product_name','item']].drop_duplicates()
product_lookup[product_lookup.product_name=='Organic Hass Avocado']

,product_id,product_name,item
1405646,47209,Organic Hass Avocado,952


In [13]:
product_lookup[product_lookup.item.isin(np.argsort(item_similarity[952])[-11:-1])]\
    .drop_duplicates()

,product_id,product_name,item
7098624,5876,Organic Lemon,115
2762349,21137,Organic Strawberries,425
8142399,26209,Limes,532
2402823,31717,Organic Cilantro,637
357827,21903,Organic Baby Spinach,445
1039807,13176,Bag of Organic Bananas,258
1163581,27966,Organic Raspberries,568
7224257,30391,Organic Cucumber,611
5590619,22935,Organic Yellow Onion,462
7561582,24964,Organic Garlic,506


# Which products has user 10370 bought?

In [14]:
train_data[train_data.user_id==10370]

,user_id,product_id,product_name,user,item
7019853,10370,5876,Organic Lemon,232,115
21169423,10370,14633,Veggie Chips,232,293
7884865,10370,38777,Organic Green Seedless Grapes,232,770
24136293,10370,10621,Organic Purple Kale,232,207
7514166,10370,24964,Organic Garlic,232,506
26173817,10370,49111,Original Instant Oatmeal,232,986
13338961,10370,32691,Vanilla Ice Cream,232,660
13264448,10370,43789,Organic Basil,232,872
18742533,10370,40002,Strawberry Ice Cream,232,796
1846787,10370,24852,Banana,232,504


# Now it's time to predict using our item-to-item similarity matrix
- A prediction can be made by (dot) multiplying the simularity matrix (#Items x #Items) by someone's profile (1 x #Items) <br>

$
\pmatrix{1.0 & 0.2 & 0.5 & 0.1 \\ 
         0.2 & 1.0 & 0.9 & 0.3 \\
         0.5 & 0.9 & 1.0 & 0.3 \\
         0.1 & 0.3 & 0.3 & 1.0} \pmatrix{0 \\ 1 \\ 1 \\ 0}  = 
         \pmatrix{0.7 \\ 1.9 \\ 1.9 \\ 0.6} \rightarrow \pmatrix{ \#1 \\ \texttt{not recommended} \\ \texttt{not recommended} \\ \#2}
$

- Grab user 10370 from the train tranactions and predict 20 items to recommend to this user.

In [15]:
def make_predictions(user_id):
    # find normalized user_id
    user_normalized = user_normalizer.lookup[user_id]
    # Dot product, followed by argsort. 
    # Multiplied by -1 such that the item with highest scores come first out of argsort. 
    predictions = np.argsort(-1*np.dot(item_similarity,train_data_matrix[user_normalized,:]))
    bought_products_train = np.where(train_data_matrix[user_normalized,:])[0] # find items in train
    # remove items already bought in train set
    valid_predictions =[]
    for prediction in predictions:
        if prediction not in bought_products_train:
            valid_predictions.append(prediction)
    return valid_predictions

In [16]:
predictions = make_predictions(10370)[:20]
predictions

[952,
 445,
 462,
 258,
 899,
 882,
 813,
 550,
 686,
 637,
 957,
 962,
 167,
 934,
 350,
 161,
 210,
 442,
 836,
 979]

# How well did you do?

In [17]:
user_normalized = user_normalizer.lookup[10370] # Find normalized user_id

bought_products_train = np.where(train_data_matrix[user_normalized,:])[0]
bought_products_test  = np.where(test_data_matrix[user_normalized,:])[0]
product_lookup[product_lookup.item.isin(bought_products_train)]

,product_id,product_name,item
7098624,5876,Organic Lemon,115
2762349,21137,Organic Strawberries,425
8142399,26209,Limes,532
19160225,38739,Hass Avocado,767
7443471,7021,Organic Tomato Paste,132
10198077,17948,Frozen Organic Wild Blueberries,357
7674547,19057,Organic Large Extra Fancy Fuji Apple,389
3482738,20995,Organic Broccoli Florets,422
13998046,23165,Organic Leek,467
1888042,24852,Banana,504


### Items bought in test set

In [18]:
product_lookup[product_lookup.item.isin(bought_products_test)]

,product_id,product_name,item
15174494,5450,Small Hass Avocado,101
18627627,25588,Seedless Small Watermelon,517
10264083,22035,Organic Whole String Cheese,449
14234017,34243,Organic Baby Broccoli,690
9079988,8277,Apple Honeycrisp Organic,161
3726821,8021,100% Recycled Paper Towels,153
5590619,22935,Organic Yellow Onion,462
11590785,44359,Organic Small Bunch Celery,882
13206214,13829,Mild Salsa,276
18967656,36695,Organic Hothouse Cucumbers,730


### Products recommended

In [19]:
product_lookup[product_lookup.item.isin(predictions[:20])]

,product_id,product_name,item
4483330,8518,Organic Red Onion,167
2402823,31717,Organic Cilantro,637
11926926,34126,Organic Italian Parsley Bunch,686
100106,17794,Carrots,350
8876749,21616,Organic Baby Arugula,442
357827,21903,Organic Baby Spinach,445
10545290,27104,Fresh Cauliflower,550
11981911,45007,Organic Zucchini,899
2523965,47766,Organic Avocado,962
1039807,13176,Bag of Organic Bananas,258


### Recommended correctly

In [20]:
products_in_predictions_and_test = \
    set(predictions[:20]).intersection(set(bought_products_test))

In [21]:
product_lookup[product_lookup.item.isin(products_in_predictions_and_test)]

,product_id,product_name,item
9079988,8277,Apple Honeycrisp Organic,161
5590619,22935,Organic Yellow Onion,462
11590785,44359,Organic Small Bunch Celery,882


## Is this good?

# Metrics for recommendations

|||
|-----------------:|-------------|--|
| precision  | % of correct rec.    | dependent on k|
| recall    | % of correct from test| dependent on k|
| f1 score  | harmonic mean of precision and recall    | dependent on k |
| mean average precision | front loading correct recommendations  | less dependend on k | 
| mean reciprical rank  | harmonic mean of rank |less dependent on k | 
|||

Calculate precision, recall and f1 score for k = 20 for user 10370

In [22]:
recommendations = make_predictions(10370)[:20]
bought_products_test  = np.where(test_data_matrix[user_normalized,:])[0]

precision = len(set(recommendations).intersection(set(bought_products_test)))/len(recommendations)
recall   = len(set(recommendations).intersection(set(bought_products_test)))/len(bought_products_test)
f1_score = 2*precision*recall/(precision+recall)
print('precision', precision)
print('recall   ', recall)
print('f1_score ', f1_score)

precision 0.15
recall    0.21428571428571427
f1_score  0.1764705882352941


<center>![](https://media.giphy.com/media/a8749TBnyEIY8/giphy.gif)</center>

# Model-based recommender


* **Before**: Recommendations based on similarity.

* **Now**: We model the observed interactions as the product of item features $x^{(i)}$ and user features $\theta^{(u)}$, such that <br>
$(\theta^{(u)})^T x^{(i)} \approx y^{(u,i)} $<br>
holds for all items

<center><img src="resources/ALS.png" width="900"></center>

With the trained model, that is with the learned user and product features, we can then compute interactions for a given user and for unseen given products.

### But what features to use?

## Motivation: content based recommendation
<center><img src="resources/content.png" width="700"></center>

## motivation: content based recommendation
| Product \ User| 0| 1| 2| 3|$|$| $x_1$ (organic)  | $x_2$ (vitamins) | $x_3$ (proteine) |$|$|item_vector | 
|:--------      |--|--|--|--|-  |----              |----   |---|--|----------------------------------------------|
| Spaghetti     | 1    | 0  | 0     | 0    |$|$| 0.5 | 0.2 | 0.1 |$|$|$ i^{(0)}=\pmatrix{ 0.5, 0.2, 0.1}$
| ground beef   | 0    | 1  | 1     | 1    |$|$| 0.3 | 0.6 | 0.9 |$|$|$i^{(1)}=\pmatrix{ 0.3, 0.6, 0.9}$
| tomato        | 1    | 1  | 0     | 1    |$|$| 1   | 0.7 | 0.3 |$|$|$i^{(2)}=\pmatrix{ 1.0, 0.7, 0.3}$
| Bell pepper   | 1    | 1  | 1     | 1    |$|$| 1   | 0.9 | 0.4 |$|$|$i^{(3)}=\pmatrix{ 1.0, 0.9, 0.4}$
| White rice    | 0    | 1  | 1     | 0    |$|$| 0.5 | 0.1 | 0.1 |$|$|$i^{(4)}=\pmatrix{ 0.5, 0.1, 0.1}$

* Given $x^{(i)}$ we can optimize for user vectors, $\theta^{(u)}$, such that when multiplied with the item vectors, we best approximate the observed purchases $y^{(u,i)}$. 
* We can do this by solving:<br>
$
\underset{\theta^{(u)}}{\min{}}
\frac{1}{2} \sum_{j:r(i,j)=1} ((\theta^{(j)})^T x^{(i)} - y^{(i,j)})^2  + 
\frac{\lambda}{2} \sum_{k=1}^n (\theta_k^{(u)})^2
$

* Given $x^{(i)}$ we can optimize for user vectors, $\theta^{(u)}$, such that when multiplied with the item vectors, we best approximate the observed purchases $y^{(u,i)}$. 
* We can do this by solving:<br>
$
\underset{\theta^{(u)}}{\min{}}
\frac{1}{2} \sum_{j:r(i,j)=1} ((\theta^{(j)})^T x^{(i)} - y^{(i,j)})^2  + 
\frac{\lambda}{2} \sum_{k=1}^n (\theta_k^{(u)})^2
$

* However, are the features that we used optimal to explain the observed interactions?
* Subsequently, given user vectors $\theta^{(u)}$ optimize for item vectors $x^{(i)}$
* Alternating Least Squares: 
$\theta \rightarrow x \rightarrow \theta \rightarrow x  \rightarrow \theta \rightarrow x\rightarrow \dots$

* There exist many packages that implement this algorithm, for example `implicit`

In [23]:
from implicit.als import AlternatingLeastSquares
from scipy import sparse

model = AlternatingLeastSquares(factors=20)

user_item_train = sparse.csr_matrix(train_data_matrix)

model.fit(user_item_train.T)

recommendations = model.recommend(user_normalizer.lookup[10370],user_item_train,\
                      N=25,\
                filter_items=list(user_item_train[user_normalizer.lookup[125981]].indices)) # To remove train items

recommendations = [x[0] for x in recommendations[:20]]

In [24]:
product_lookup[product_lookup.item.isin(recommendations)]

,product_id,product_name,item
2402823,31717,Organic Cilantro,637
18007198,16759,Organic Carrot Bunch,330
15515824,27086,Half & Half,549
10545290,27104,Fresh Cauliflower,550
11981911,45007,Organic Zucchini,899
2523965,47766,Organic Avocado,962
3420389,27845,Organic Whole Milk,567
14234017,34243,Organic Baby Broccoli,690
1405646,47209,Organic Hass Avocado,952
9079988,8277,Apple Honeycrisp Organic,161


### Correct recommendations by implicit ALS

In [25]:
product_lookup[product_lookup.item.isin(set(recommendations).intersection(set(bought_products_test)))]

,product_id,product_name,item
14234017,34243,Organic Baby Broccoli,690
9079988,8277,Apple Honeycrisp Organic,161
11590785,44359,Organic Small Bunch Celery,882


# Neural network based models

* Recently, advanced deep learning frameworks
* More freedom with development
* Better training possibilities

## Youtube Recommender

<center><img src="resources/neuralnet.png" width="600"></center>

# Conclusion

* Recommendations everywhere
* Simple algorithms work well
* Trend to use implicit feedback only
* Active research area, especially focussed on deep learning techniques